In [1]:
import cv2
import numpy as np
import speech_recognition as sr
import pyttsx3
import time
import threading

# Show the different modes of control
print("Welcome to the Lane Detection System!")
print("\n Choose a mode: \n 1. Auto Lane Detection \n 2. Voice-controlled Lane Detection")
mode=input("Enter 1 or 2: ").strip()

#declaring a global variable to track lane drift
drift_out_of_lane= False

# Initialize the text-to-speech engine
engine= pyttsx3.init()
engine.setProperty('rate', 150)


# Function to speak text
def speak(text):
    engine.say(text)
    engine.runAndWait()


# Function to take voice commands
def listen_commands():
    global drift_out_of_lane
    recognizer = sr.Recognizer()
    mic=sr.Microphone()

    with mic as source:
        recognizer.adjust_for_ambient_noise(source)
    
    while True:
        with mic as source:
            print("Listening for commands...")
            audio = recognizer.listen(source)
        
        try:
            command = recognizer.recognize_google(audio).lower()
            print(f"Command received: {command}")
            
            if "drift out of lane" in command:
                drift_out_of_lane = True
                speak("Warning: Lane Departure Detected!")
            elif "stop drifting" in command:
                drift_out_of_lane = False
                speak("You are back in lane")
        
        except sr.UnknownValueError:
            print("Could not understand the command.")
        except sr.RequestError as e:
            print(f"Could not request results; {e}")

# start voice listener in the bg thread
if mode == "2":
    threading.Thread(target=listen_commands, daemon=True).start()#this will run once you chose the voice-controlled mode


#uplaoad a video file for lane detection
# 'test_video.mp4' This can be replaced ,with your video file path
cap=cv2.VideoCapture('lane_video_2.mp4')



# Function to preprocess the frame, returning the frame after converting to gray and then bluring the noise
def preprocess_frame(frame):
    # convert to gray scale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #applying gaussian blur to remove noise from the background
    blur = cv2.GaussianBlur(gray,(5,5),0)

    return blur


# cany helps to detect the edges in the image, it helps us by changing the brightness levels easily
def detect_edges(blur):
    # applying canny edge detection
    edges = cv2.Canny(blur, 50, 150)# 50 is the lower threshold and 150 is the upper threshold
    # dilating the edges to make them more pronounced
    return edges


# we need to have a roi to focus on the area of interest, so we can define a polygon
def region_of_interest(edges):
    height, width = edges.shape

    # defining a polygon to focus on the area of interest
    polygon = np.array([[(0, height), (width, height), (int(width*0.5), int(height*0.6))]], dtype=np.int32)

    mask = np.zeros_like(edges)#we are making a mask of the same size as the edges
    # filling the polygon in the mask with white color
    # this will create a mask that will only allow the region of interest to be visible
    # cv2.fillPoly() is used to fill the polygon with white color
    cv2.fillPoly(mask, polygon, 255)
    cropped_edges= cv2.bitwise_and(edges, mask)
    # applying bitwise and operation to the edges and the mask to get the region of interest
    # this will create a new image that only contains the region of interest
    return cropped_edges



#use hough transform to detect the lanes in the image
def detect_lines(cropped_edges):
    lines=cv2.HoughLinesP(
        cropped_edges,
        rho=2,#precision of distance values you want
        theta=np.pi/180,#angle resolutions are to be checked
        threshold=50,#minimum number of votes to be considered as a line
        # if the number of votes is less than this threshold, the line will not be detected
        lines=np.array([]),#detected lines are stored in this array
        minLineLength=100,#minimum length of a line to be detected -> ignore very short lines
        maxLineGap=5# maximum gap between two lines to be considered as a single line -> allow small gaps in lines to be connected
    )
    return lines

# we need to draw lines on a blank image

def draw_lines(frame, lines):
    line_image = np.zeros_like(frame)  # create a blank image with the same size as the frame

    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line: # 
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)  # draw the green line on the blank image

    return line_image
#x1,y1 and x2,y2 are the start and end points of two lines respectively
# we need to overlay the lines on the original frame


def is_vehicle_in_lane(lines, frame_width, threshold=50):
    """
    Checks whether the car is centered between the detected lanes.

    Returns:
        - True if vehicle is in lane
        - False if vehicle is drifting
    """
    if lines is None:
        return False

    left_x = []
    right_x = []

    for line in lines:
        for x1, y1, x2, y2 in line:
            if (x2 - x1) == 0:
                continue
            slope = (y2 - y1) / (x2 - x1)
            if slope < -0.5:
                left_x += [x1, x2]
            elif slope > 0.5:
                right_x += [x1, x2]

    if not left_x or not right_x:
        return False

    # Estimate center of the lane
    lane_center = (max(left_x) + min(right_x)) // 2
    car_center = frame_width // 2
    offset = abs(car_center - lane_center)

    return offset < threshold


def draw_lane_area(frame, lines):
    left_lines = []
    right_lines = []

    if lines is None:
        return np.zeros_like(frame)  # Return a blank image safely

    for line in lines:
        for x1, y1, x2, y2 in line:
            if (x2 - x1) == 0:  # Avoid division by zero
                continue
            slope = (y2 - y1) / (x2 - x1)

            if slope < -0.5:  # left lane
                left_lines.append((x1, y1, x2, y2))
            elif slope > 0.5:  # right lane
                right_lines.append((x1, y1, x2, y2))

    lane_image = np.zeros_like(frame)

    for x1, y1, x2, y2 in left_lines:
        cv2.line(lane_image, (x1, y1), (x2, y2), (0, 255, 0), 8)
    for x1, y1, x2, y2 in right_lines:
        cv2.line(lane_image, (x1, y1), (x2, y2), (0, 0, 255), 8)

    return lane_image
# This function will draw the lane area on the frame
# It separates left and right lanes based on slope and draws them in different colors (green for left, red for right)

# Real time instruction to avoid moving out of lane


def show_instruction(frame, lines, in_lane=True):
    if lines is None:
        instruction = ""
        color = (255, 0, 0)
    elif not in_lane:
        instruction = "Drifting from lane!"
        color = (0, 0, 255)
    else:
        instruction = "Stay in the lane"
        color = (0, 255, 0)

    cv2.putText(frame, instruction, (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3, cv2.LINE_AA)



# now combine the orginal video with the detected lines
def overlay_lines(frame, line_image):
    return cv2.addWeighted(frame, 0.8, line_image, 1, 1) # overlay the lines on the original frame
# 0.8 is the weight of the original frame and 1 is the weight of the line image
# 1 is the weight of the line image and 1 is the gamma value
# gamma value is used to adjust the brightness of the image
# 0.8 means original frame is 80% and line image is 20%



while cap.isOpened():
    ret,frame=cap.read()
    if not ret:
        break
    height, width = frame.shape[:2]
    display_frame=frame.copy()

    if mode == "2":

        overlay=frame.copy()
    #draw stimulated lane area

        if drift_out_of_lane:
            pts=np.array([[width//2-200, height], 
                      [width//2+200, height], 
                      [width//2+150, int(height*0.6)],
                      [width//2 -250 , int(height*0.6)]],
                        np.int32)
            cv2.putText(overlay,"Lane Departure Detected!", (50,50),cv2.FONT_HERSHEY_SIMPLEX,1.2, (0,0,255), 3)
        else:
            pts=np.array([[width//2-100, height], 
                      [width//2+100, height], 
                      [width//2+50, int(height*0.6)],
                      [width//2 -150 , int(height*0.6)]],
                        np.int32)
            cv2.putText(overlay,"You are in lane", (50,50),cv2.FONT_HERSHEY_SIMPLEX,1.2, (0,255,0), 3)
        pts=pts.reshape((-1,1,2))
        cv2.fillPoly(overlay, [pts], (255, 255, 0)) #light blue
        display_frame=cv2.addWeighted(display_frame, 0.8, overlay, 0.4, 1)
    
    else:

        blur = preprocess_frame(frame)
        edges = detect_edges(blur)
        cropped = region_of_interest(edges)
        lines = detect_lines(cropped)

        
        # Draw lines
        line_image = draw_lines(display_frame, lines)
        
        if lines is not None:
        # Overlay both lane + line drawings
            lane_image = draw_lane_area(display_frame, lines)

            display_frame = overlay_lines(display_frame, lane_image)

        # Show instruction (like "Stay in Lane")
        in_lane = is_vehicle_in_lane(lines, width)
        show_instruction(display_frame, lines, in_lane)

        # Mode label
        cv2.putText(display_frame, "Auto Lane Detection", (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 0), 3)
        
    cv2.imshow('Lane Detection',display_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()
# this will release the video capture and close all the windows

    


Welcome to the Lane Detection System!

 Choose a mode: 
 1. Auto Lane Detection 
 2. Voice-controlled Lane Detection
